In [73]:
import faker 

fake = faker.Faker()

print(fake.email())
print(fake.address())
print(fake.company())



jennifer35@example.com
6379 Rojas Mountain Apt. 779
Holmesberg, AZ 13796
Stein Inc


In [74]:
import json
#Genrate a user
users = []
for i in range(1000):
    user ={
        "username": fake.unique.user_name(),
        "name": fake.name(),
        "email": fake.email(),
        "address": fake.address(),
        "age": fake.random_int(min=18, max=90)
    }
    users.append(user)

#Save in JSON
with open("users.json", "w") as file:
    json.dump(users, file, indent=4)

In [75]:
import random
from datetime import date, timedelta

def get_start_end_dates():
    duration = random.randint(1, 2 * 365)
    offset = random.randint(-365, 365)
    start = date.today() - timedelta(days=offset)
    end = start + timedelta(days=duration)
    
    return start.strftime("%Y%m%d"), end.strftime("%Y%m%d")

print(get_start_end_dates())

def get_age_range():
    age = random.randrange(20, 46, 5)
    diff = random.randrange(5, 26, 5)

    return f"{age}-{age + diff}"

print(get_age_range())

def get_currency():
    return random.choice(("GBP", "USD", "EUR"))

#Create campaign name
def get_campaign_name():
    name = get_start_end_dates()[0] + "_" + get_start_end_dates()[1] + "_" + get_age_range() + "_" + get_currency()
    return name

print(get_campaign_name())

('20250530', '20270512')
35-60
20251126_20280630_25-30_USD


In [76]:
#Generate the data for the campaign
def get_campaign_data():
    name = get_campaign_name()
    budget = random.randint(10**3, 10**6)
    spent = random.randint(10**2, budget)

    return{
        "cmp_name": name, 
        "cmp_bgt": budget,
        "cmp_spent": spent, 
    }

print(get_campaign_data())

def get_raw_data(users):
    raw_data = []
    for user in users:
        campaigns = []
        for i in range(random.randint(5, 10)):
            campaign_data = get_campaign_data()
            dates = campaign_data["cmp_name"].split("_")[2].split("-")
            min_age = int(dates[0])
            max_age = int(dates[1])
            if(user["age"] >= min_age and user["age"] <= max_age):
                campaigns.append(campaign_data)

        raw_data.append({"user": user, "campaign": campaigns})
    return raw_data

raw_data = get_raw_data(users)
with open("raw_data.json", "w") as file:
    json.dump(raw_data, file, indent=4)

campaign_data = []
for data in raw_data:
    for campaign in data["campaign"]:
        campaign["user"] = data["user"]
        campaign_data.append(campaign)

with open("campaigns_data.json", "w") as file:
    json.dump(campaign_data, file, indent=4)

{'cmp_name': '20260902_20270630_30-55_USD', 'cmp_bgt': 378124, 'cmp_spent': 90706}


In [77]:
import pandas as pd

df = pd.read_json("campaigns_data.json")

df.head(10)



,cmp_name,cmp_bgt,cmp_spent,user
0,20260718_20270822_20-25_GBP,540607,69578,"{'username': 'dflynn', 'name': 'Kelly Martin',..."
1,20260113_20280213_20-40_EUR,703404,701923,"{'username': 'dflynn', 'name': 'Kelly Martin',..."
2,20260620_20270422_40-60_GBP,992067,864733,"{'username': 'leemaria', 'name': 'Julia King',..."
3,20250131_20251109_45-65_USD,499659,26329,"{'username': 'leemaria', 'name': 'Julia King',..."
4,20260509_20270201_30-55_EUR,379816,67617,"{'username': 'leemaria', 'name': 'Julia King',..."
5,20260927_20270524_25-50_GBP,68072,8984,"{'username': 'leemaria', 'name': 'Julia King',..."
6,20260311_20250201_20-30_USD,888378,683184,"{'username': 'garzakimberly', 'name': 'Daniel ..."
7,20260803_20261129_25-50_USD,743537,606711,"{'username': 'garzakimberly', 'name': 'Daniel ..."
8,20250707_20260418_40-50_EUR,578245,59487,"{'username': 'andersonclaire', 'name': 'Nichol..."
9,20260613_20260115_45-65_GBP,451287,376153,"{'username': 'andersonclaire', 'name': 'Nichol..."


In [78]:
#Return the data types of all columns
print(df.dtypes)

cmp_name     object
cmp_bgt       int64
cmp_spent     int64
user         object
dtype: object


In [79]:
#Filter the data
print(df.query("cmp_bgt < 100000 and cmp_spent > 50000"))

                         cmp_name  cmp_bgt  cmp_spent  \
81    20251003_20250219_35-60_USD    85551      80797   
204   20250507_20250730_25-50_USD    89130      74445   
207   20251102_20261024_30-40_EUR    69357      65635   
217   20251125_20270704_20-35_USD    94070      78132   
223   20250521_20270807_25-30_EUR    81702      73077   
226   20241124_20270402_35-55_EUR    92412      54426   
274   20261020_20270113_40-60_EUR    54318      53546   
484   20250801_20270604_20-30_EUR    96024      88070   
516   20260617_20270413_20-45_USD    95701      88729   
592   20260412_20280402_20-35_USD    92602      83265   
708   20250522_20270404_20-25_GBP    92808      68702   
819   20251214_20250802_30-40_GBP    54566      51722   
840   20250411_20280301_45-70_GBP    67217      61102   
910   20251114_20261129_20-40_USD    77597      76892   
961   20250704_20270918_40-55_EUR    85850      64200   
1002  20251117_20280127_20-25_USD    76213      65511   
1126  20261104_20260601_25-30_E

In [80]:
#Get the campaign with max spent
index = df["cmp_spent"].idxmax()
df.loc[index]


cmp_name                           20260621_20270110_30-35_USD
cmp_bgt                                                 991077
cmp_spent                                               978726
user         {'username': 'jasminemoses', 'name': 'Joshua J...
Name: 1284, dtype: object

In [81]:
df.sort_values(by=["cmp_spent"], ascending=False).head(1)

,cmp_name,cmp_bgt,cmp_spent,user
1284,20260621_20270110_30-35_USD,991077,978726,"{'username': 'jasminemoses', 'name': 'Joshua J..."


In [82]:
#Add a new column 
df["cmp_remaining_budget"] = df["cmp_bgt"] - df["cmp_spent"]

#Get the start date
start_date = df["cmp_name"].str.split("_").str[0]

df["cmp_start_date"] = pd.to_datetime(start_date)

#TODO
#df["cmp_days_running"] 

df.head(10)

df.describe()

,cmp_bgt,cmp_spent,cmp_remaining_budget,cmp_start_date
count,1591.000000,1591.000000,1591.000000,1591
mean,486877.118165,242616.450660,244260.667505,2025-11-15 17:41:40.314267904
min,1936.000000,294.000000,59.000000,2024-11-19 00:00:00
25%,241106.500000,70980.000000,72665.000000,2025-05-17 00:00:00
50%,467562.000000,176174.000000,180457.000000,2025-11-06 00:00:00
75%,737873.000000,363238.000000,369424.000000,2026-05-19 00:00:00
max,999743.000000,978726.000000,961156.000000,2026-11-19 00:00:00
std,286974.937409,215711.982702,214467.027011,NaN


In [83]:
df.to_csv("data.csv")

In [85]:
df.to_excel("data.xlsx")